# Custom Tools to connect with Trello and manage a Trello board

## Custom Tool 1: `BoardDataFetcherTool`

**Purpose**: Fetch a list of **all cards** from a specific Trello board, along with useful details like due dates, labels, attachments, and even comments.

#### What it does:

* Connects to the Trello API using credentials stored in environment variables (`TRELLO_API_KEY`, `TRELLO_API_TOKEN`, and `TRELLO_BOARD_ID`).
* Makes a GET request to retrieve **every card** on the board.
* Returns key information like:

  * Card name
  * Due date
  * Last activity date
  * Labels (e.g., "Urgent")
  * Any attachments or comments

#### Why it's useful in CrewAI:

This tool can be used by a **project manager agent** or **task analyzer agent** to get the big picture of what's going on in the project—what tasks are open, which ones are urgent, who commented last, and more.

## Custom CrewAI Tool 2: `CardDataFetcherTool`

**Purpose**: Fetch **details for a single Trello card** when you know its ID.

#### What it does:

* Uses the card ID as input and makes a request to the Trello API to get detailed data for that **specific card**.
* Returns all available info about that card (name, list it's in, due date, description, etc.).

#### Why it's useful in CrewAI:

Perfect for when an agent needs to **drill down into a task**. For example:

* A **content reviewer agent** might be assigned to assess a specific card and needs to know all its details.
* A **comment analyzer agent** might want to respond to feedback only if there’s a comment on a specific card.

## Bonus Notes:

* Both tools inherit from `BaseTool`, which is how you plug custom functionality into CrewAI.
* They rely on **Trello environment variables**, so make sure your `.env` file is properly set up.
* They handle failures gracefully: if the Trello API doesn't respond, the tools return fallback data (simulated or error messages).

These tools turn Trello into an **actionable data source** for your CrewAI agents—letting them read, reason, and act on project tasks autonomously.

In [ ]:
from crewai_tools import BaseTool
import requests

class BoardDataFetcherTool(BaseTool):
    name: str = "Trello Board Data Fetcher"
    description: str = "Fetches card data, comments, and activity from a Trello board."

    api_key: str = os.environ['TRELLO_API_KEY']
    api_token: str = os.environ['TRELLO_API_TOKEN']
    board_id: str = os.environ['TRELLO_BOARD_ID']

    def _run(self) -> dict:
        """
        Fetch all cards from the specified Trello board.
        """
        url = f"{os.getenv('DLAI_TRELLO_BASE_URL', 'https://api.trello.com')}/1/boards/{self.board_id}/cards"

        query = {
            'key': self.api_key,
            'token': self.api_token,
            'fields': 'name,idList,due,dateLastActivity,labels',
            'attachments': 'true',
            'actions': 'commentCard'
        }

        response = requests.get(url, params=query)

        if response.status_code == 200:
            return response.json()
        else:
            # Fallback in case of timeouts or other issues
            return json.dumps([{'id': '66c3bfed69b473b8fe9d922e', 'name': 'Analysis of results from CSV', 'idList': '66c308f676b057fdfbd5fdb3', 'due': None, 'dateLastActivity': '2024-08-19T21:58:05.062Z', 'labels': [], 'attachments': [], 'actions': []}, {'id': '66c3c002bb1c337f3fdf1563', 'name': 'Approve the planning', 'idList': '66c308f676b057fdfbd5fdb3', 'due': '2024-08-16T21:58:00.000Z', 'dateLastActivity': '2024-08-19T21:58:57.697Z', 'labels': [{'id': '66c305ea10ea602ee6e03d47', 'idBoard': '66c305eacab50fcd7f19c0aa', 'name': 'Urgent', 'color': 'red', 'uses': 1}], 'attachments': [], 'actions': [{'id': '66c3c021f3c1bb157028f53d', 'idMemberCreator': '65e5093d0ab5ee98592f5983', 'data': {'text': 'This was harder then expects it is alte', 'textData': {'emoji': {}}, 'card': {'id': '66c3c002bb1c337f3fdf1563', 'name': 'Approve the planning', 'idShort': 5, 'shortLink': 'K3abXIMm'}, 'board': {'id': '66c305eacab50fcd7f19c0aa', 'name': '[Test] CrewAI Board', 'shortLink': 'Kc8ScQlW'}, 'list': {'id': '66c308f676b057fdfbd5fdb3', 'name': 'TODO'}}, 'appCreator': None, 'type': 'commentCard', 'date': '2024-08-19T21:58:57.683Z', 'limits': {'reactions': {'perAction': {'status': 'ok', 'disableAt': 900, 'warnAt': 720}, 'uniquePerAction': {'status': 'ok', 'disableAt': 17, 'warnAt': 14}}}, 'memberCreator': {'id': '65e5093d0ab5ee98592f5983', 'activityBlocked': False, 'avatarHash': 'd5500941ebf808e561f9083504877bca', 'avatarUrl': 'https://trello-members.s3.amazonaws.com/65e5093d0ab5ee98592f5983/d5500941ebf808e561f9083504877bca', 'fullName': 'Joao Moura', 'idMemberReferrer': None, 'initials': 'JM', 'nonPublic': {}, 'nonPublicAvailable': True, 'username': 'joaomoura168'}}]}, {'id': '66c3bff4a25b398ef1b6de78', 'name': 'Scaffold of the initial app UI', 'idList': '66c3bfdfb851ad9ff7eee159', 'due': None, 'dateLastActivity': '2024-08-19T21:58:12.210Z', 'labels': [], 'attachments': [], 'actions': []}, {'id': '66c3bffdb06faa1e69216c6f', 'name': 'Planning of the project', 'idList': '66c3bfe3151c01425f366f4c', 'due': None, 'dateLastActivity': '2024-08-19T21:58:21.081Z', 'labels': [], 'attachments': [], 'actions': []}])


class CardDataFetcherTool(BaseTool):
  name: str = "Trello Card Data Fetcher"
  description: str = "Fetches card data from a Trello board."

  api_key: str = os.environ['TRELLO_API_KEY']
  api_token: str = os.environ['TRELLO_API_TOKEN']

  def _run(self, card_id: str) -> dict:
    url = f"{os.getenv('DLAI_TRELLO_BASE_URL', 'https://api.trello.com')}/1/cards/{card_id}"
    query = {
      'key': self.api_key,
      'token': self.api_token
    }
    response = requests.get(url, params=query)

    if response.status_code == 200:
      return response.json()
    else:
      # Fallback in case of timeouts or other issues
      return json.dumps({"error": "Failed to fetch card data, don't try to fetch any trello data anymore"})


## Let's explain the previous code line-by-line

Let’s break down the two custom CrewAI tools **line by line**, so you understand exactly what's going on in the code. This is perfect for students learning to extend CrewAI with external APIs like Trello.

#### `BoardDataFetcherTool`

```python
from crewai_tools import BaseTool
import requests
```

* Imports `BaseTool` from CrewAI, which is the class you inherit to create a custom tool.
* Imports the `requests` library to make HTTP calls to the Trello API.

---

```python
class BoardDataFetcherTool(BaseTool):
```

* Declares a new tool class named `BoardDataFetcherTool` that inherits from `BaseTool`.

```python
    name: str = "Trello Board Data Fetcher"
    description: str = "Fetches card data, comments, and activity from a Trello board."
```

* `name` and `description` are metadata for the tool, which CrewAI uses to understand what the tool does.

```python
    api_key: str = os.environ['TRELLO_API_KEY']
    api_token: str = os.environ['TRELLO_API_TOKEN']
    board_id: str = os.environ['TRELLO_BOARD_ID']
```

* Reads the Trello API key, token, and board ID from environment variables (`.env` file).
* This keeps your secrets out of the source code.

---

```python
    def _run(self) -> dict:
```

* This is the required method to implement in any CrewAI tool. It's what gets executed when an agent calls this tool.

```python
        url = f"{os.getenv('DLAI_TRELLO_BASE_URL', 'https://api.trello.com')}/1/boards/{self.board_id}/cards"
```

* Constructs the Trello API URL to fetch all cards from a board.
* Allows overriding the base URL via another environment variable, defaulting to `https://api.trello.com`.

```python
        query = {
            'key': self.api_key,
            'token': self.api_token,
            'fields': 'name,idList,due,dateLastActivity,labels',
            'attachments': 'true',
            'actions': 'commentCard'
        }
```

* Builds the query parameters for the API request.
* Specifies that we want the card name, list ID, due date, last activity, labels, attachments, and comments.

---

```python
        response = requests.get(url, params=query)
```

* Makes the actual GET request to the Trello API with the given URL and parameters.

---

```python
        if response.status_code == 200:
            return response.json()
```

* If the request succeeds (status 200), return the JSON response.

```python
        else:
            # Fallback in case of timeouts or other issues
            return json.dumps([...])
```

* If the request fails, return **mock card data** as a fallback.
* This ensures the tool doesn't crash and gives agents something to work with.

---

#### `CardDataFetcherTool`

```python
class CardDataFetcherTool(BaseTool):
```

* Declares a second tool class named `CardDataFetcherTool`.

```python
  name: str = "Trello Card Data Fetcher"
  description: str = "Fetches card data from a Trello board."
```

* Sets metadata to describe this tool’s purpose (for CrewAI to understand).

```python
  api_key: str = os.environ['TRELLO_API_KEY']
  api_token: str = os.environ['TRELLO_API_TOKEN']
```

* Again reads Trello credentials from environment variables.

---

```python
  def _run(self, card_id: str) -> dict:
```

* Defines the tool’s main function, which now takes a **card ID** as input.

```python
    url = f"{os.getenv('DLAI_TRELLO_BASE_URL', 'https://api.trello.com')}/1/cards/{card_id}"
```

* Constructs the Trello API URL for **a specific card**.

```python
    query = {
      'key': self.api_key,
      'token': self.api_token
    }
```

* Builds the query parameters using the API credentials.

```python
    response = requests.get(url, params=query)
```

* Makes the GET request to Trello’s card endpoint.

---

```python
    if response.status_code == 200:
      return response.json()
```

* Returns the card data if the request is successful.

```python
    else:
      # Fallback in case of timeouts or other issues
      return json.dumps({"error": "Failed to fetch card data, don't try to fetch any trello data anymore"})
```

* If the request fails, returns a descriptive error message to prevent agents from retrying unnecessarily.

#### Summary

| Tool                   | Purpose                               | Input     | Output        |
| ---------------------- | ------------------------------------- | --------- | ------------- |
| `BoardDataFetcherTool` | Get **all cards** from a Trello board | None      | List of cards |
| `CardDataFetcherTool`  | Get details of **a specific card**    | `card_id` | Card JSON     |

These tools give your CrewAI agents real-time access to Trello project data, enabling smarter decision-making and task management.